In [1]:
%display latex

In [2]:
%runfile character.py

Morceau de code de François pour affichage

In [3]:
def Crache(F, deg, w=None, g=None):
    if F==0: 
        show()
    elif g==None:
        Dishout(F,deg,s,s)
    else:
        n=deg
        for mu in sorted(list(Partitions(n))):
            val=w(Scalar2(F,g(mu)))
            if val<>0:
                show((tuple(mu),val))

@cached_function
def Scalar2(U,f): 
    res=add(_c*s(_a)*(s(_b).scalar(f)) for (_a,_b),_c in tensor([s,s])(U))
    if res==None: 
        return 0*Un
    else:
        return res

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [4]:
def vandermonde(gamma):
    n = gamma.size()
    P = DiagonalPolynomialRing(QQ, n, 1, inert=1)
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

In [5]:
vandermonde(Partition([2,1]))

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

In [6]:
vandermonde(Diagram([(0,0),(2,0),(4,0)]))

-x00^4*x01^2 + x00^2*x01^4 + x00^4*x02^2 - x01^4*x02^2 - x00^2*x02^4 + x01^2*x02^4

In [7]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
v.degree()

6

In [8]:
v.parent()

Multivariate Polynomial Ring in x00, x01, x02, theta00, theta01, theta02 over Rational Field

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [9]:
def deriv(x, k=1):
    def f(p):
        return derivative(p, x, k)
    return f

def partial_derivatives(P):
    n = P._n
    r = P._r
    D = P._grading_set
    X = P.variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [deriv(X[i,k]) for k in range(n)]
    return op

def steenrod_op(P, degree=1):
    r = P._r
    D = P._grading_set
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [functools.partial(P.steenrod_op, i=i, k=degree+1)]
    return op

def merge_dict(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

Créaction du sous-espace engendré par un détermiant de type Vandermonde, ses dérivées partielles et les opérateurs de Steenrod de degré 1 et 2.

In [58]:
P = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
generator = {P._grading_set((v.degree(),)) : [P(v)]}
list_steenrod = merge_dict(steenrod_op(P,1), steenrod_op(P,2))
list_op = merge_dict(partial_derivatives(P), list_steenrod)

In [59]:
V1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 (3,): (x01*x02^2 - 1/3*x02^3,
  x00*x02^2 - 1/3*x02^3,
  x00^2*x02 - 1/3*x02^3,
  x00*x01^2 - 1/3*x02^3,
  x00^2*x01 - 1/3*x02^3,
  x01^2*x02 - 1/3*x02^3,
  x01^3 - x02^3,
  x00^3 - x02^3),
 (4,): (-1/4*x00^4 + x00^3*x01 - x01^3*x02 + 1/4*x02^4,
  -1/4*x00^4 + x00*x01^3 - x01^3*x02 + 1/4*x02^4,
  -x01^3*x02 + x01*x02^3,
  -1/4*x00^4 + 1/4*x01^4 + x00^3*x02 - x01^3*x02,
  -1/4*x00^4 + 1/4*x01^4 - x01^3*x02 + x00*x02^3,
  1/6*x00^4 - x00^2*x01^2 + x01^2*x02^2 - 1/6*x02^4,
  -x00^2*x01^2 + 1/6*x01^4 + x00^2*x02^2 - 1/6*x02^4),
 (5,): (x00^2*x01^3 - 1/2*x00*x01^4 - 1/2*x00^4*x02 + 1/2*x01^4*x02 - x01^3*x02^2 + 1/2*x00*x02^4,
  -x00^3*x01^2 + 1/2*x00*x01^4 + x00^3*x02^2 - 1/2*x00*x02^4,
  1/2*x00^4*x02 - 1/2*x01^4*x02 - x00^2*x02^3 + x01^2*x02^3,
  x00^4*x01 - x00*x01^4 - x00^4*x02 + x01^4*x02 + x00*x02^4 - x01*x02^4),
 (6,): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,)}

Projection sur les composantes isotypiques.

Prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [60]:
def isotyp(S, arg):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    
    basis = S.basis()
    result = {}
    for nu in list_partitions:
        for key, value in basis.iteritems():
            result[(key, tuple(nu))] = Subspace(value, [functools.partial(apply_young_idempotent, t=nu)]).basis()[0]
    return result

In [61]:
isotyp(V1, Partition([1,1,1]))

{((2,), (1, 1, 1)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((6,),
  (1,
   1,
   1)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((3,), (1, 1, 1)): (-1/3*x00^3 + x01*x02^2,
  -x00^3 + x02^3,
  -1/3*x00^3 + x00*x02^2,
  -1/3*x00^3 + x00^2*x02,
  -1/3*x00^3 + x00*x01^2,
  -1/3*x00^3 + x00^2*x01,
  -1/3*x00^3 + x01^2*x02,
  -x00^3 + x01^3),
 ((5,),
  (1,
   1,
   1)): (-1/2*x00^4*x01 + x00^2*x01^3 - x01^3*x02^2 + 1/2*x01*x02^4, x00^4*x01 - x00*x01^4 - 2*x00^2*x02^3 + 2*x01^2*x02^3 + x00*x02^4 - x01*x02^4, -x00^4*x02 + x01^4*x02 + 2*x00^2*x02^3 - 2*x01^2*x02^3, -1/2*x00^4*x01 + x00^3*x01^2 - x00^3*x02^2 + x00^2*x02^3 - x01^2*x02^3 + 1/2*x01*x02^4),
 ((0,), (1, 1, 1)): (1,),
 ((1,), (1, 1, 1)): (x00, x01, x02),
 ((4,), (1, 1, 1)): (x01^3*x02 + 3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3,
  x00^3*x01 - 3/2*x00^2*x01^2 + 3/2*x00^2*x02^2 - x00*x02^3,
  6*x00^2*x01^2 - x01^4 - 6*x00^2*x02^2 + x02^4,
  x00^4 - x01^4 - 6*x00^2*x02^2 + 6*x01^2*x02^2,
  -3/2*x00^2*x01^2 + x00*x01^3 + 3/2*x00^2*x02^2 - x00*x02^3,
  3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3 + x01*x02^3,
  x00^3*x02 - x00*x02^3)}

In [62]:
isotyp(V1, 3)

{((4,), (3,)): (x01^3*x02 + 3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3,
  x00^3*x01 - 3/2*x00^2*x01^2 + 3/2*x00^2*x02^2 - x00*x02^3,
  6*x00^2*x01^2 - x01^4 - 6*x00^2*x02^2 + x02^4,
  x00^4 - x01^4 - 6*x00^2*x02^2 + 6*x01^2*x02^2,
  -3/2*x00^2*x01^2 + x00*x01^3 + 3/2*x00^2*x02^2 - x00*x02^3,
  3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3 + x01*x02^3,
  x00^3*x02 - x00*x02^3),
 ((6,),
  (1,
   1,
   1)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((6,),
  (2,
   1)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((2,), (2, 1)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((0,), (1, 1, 1)): (1,),
 ((0,), (3,)): (1,),
 ((5,), (2, 1)): (-1/2*x00^4*x01 + x00^2*x01^3 - x01^3*x02^2 + 1/2*x01*x02^4,
  x00^4*x01 - x00*x01^4 - 2*x00^2*x02^3 + 2*x01^2*x02^3 + x00*x02^4 - x01*x02^4,
  -x00^4*x02 + x01^4*x02 + 2*x00^2*x02^3 - 2*x01^2*x02^3,
  -1/2*x00^4*x01 + x00^3*x01^2 - x00^3*x02^2 + x00^2*x02^3 - x01^2*x02^3 + 1/2*x01*x02^4),
 ((5,), (3,)): (-1/2*x00^4*x01 + x00^2*x01^3 - x01^3*x02^2 + 1/2*x01*x02^4,
  x00^4*x01 - x00*x01^4 - 2*x00^2*x02^3 + 2*x01^2*x02^3 + x00*x02^4 - x01*x02^4,
  -x00^4*x02 + x01^4*x02 + 2*x00^2*x02^3 - 2*x01^2*x02^3,
  -1/2*x00^4*x01 + x00^3*x01^2 - x00^3*x02^2 + x00^2*x02^3 - x01^2*x02^3 + 1/2*x01*x02^4),
 ((3,), (1, 1, 1)): (-1/3*x00^3 + x01*x02^2,
  -x00^3 + x02^3,
  -1/3*x00^3 + x00*x02^2,
  -1/3*x00^3 + x00^2*x02,
  -1/3*x00^3 + x00*x01^2,
  -1/3*x00^3 + x00^2*x01,
  -1/3*x00^3 + x01^2*x02,
  -x00^3 + x01^3),
 ((3,), (3,)): (-1/3*x00^3 + x01*x02^2,
  -x00^3 + x02^3,
  -1/3*x00^3 + x00*x02^2,
  -1/3*x00^3 + x00^2*x02,
  -1/3*x00^3 + x00*x01^2,
  -1/3*x00^3 + x00^2*x01,
  -1/3*x00^3 + x01^2*x02,
  -x00^3 + x01^3),
 ((4,), (1, 1, 1)): (x01^3*x02 + 3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3,
  x00^3*x01 - 3/2*x00^2*x01^2 + 3/2*x00^2*x02^2 - x00*x02^3,
  6*x00^2*x01^2 - x01^4 - 6*x00^2*x02^2 + x02^4,
  x00^4 - x01^4 - 6*x00^2*x02^2 + 6*x01^2*x02^2,
  -3/2*x00^2*x01^2 + x00*x01^3 + 3/2*x00^2*x02^2 - x00*x02^3,
  3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3 + x01*x02^3,
  x00^3*x02 - x00*x02^3),
 ((5,),
  (1,
   1,
   1)): (-1/2*x00^4*x01 + x00^2*x01^3 - x01^3*x02^2 + 1/2*x01*x02^4, x00^4*x01 - x00*x01^4 - 2*x00^2*x02^3 + 2*x01^2*x02^3 + x00*x02^4 - x01*x02^4, -x00^4*x02 + x01^4*x02 + 2*x00^2*x02^3 - 2*x01^2*x02^3, -1/2*x00^4*x01 + x00^3*x01^2 - x00^3*x02^2 + x00^2*x02^3 - x01^2*x02^3 + 1/2*x01*x02^4),
 ((0,), (2, 1)): (1,),
 ((2,), (1, 1, 1)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((1,), (1, 1, 1)): (x00, x01, x02),
 ((1,), (2, 1)): (x00, x01, x02),
 ((1,), (3,)): (x00, x01, x02),
 ((6,),
  (3,)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((3,), (2, 1)): (-1/3*x00^3 + x01*x02^2,
  -x00^3 + x02^3,
  -1/3*x00^3 + x00*x02^2,
  -1/3*x00^3 + x00^2*x02,
  -1/3*x00^3 + x00*x01^2,
  -1/3*x00^3 + x00^2*x01,
  -1/3*x00^3 + x01^2*x02,
  -x00^3 + x01^3),
 ((4,), (2, 1)): (x01^3*x02 + 3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3,
  x00^3*x01 - 3/2*x00^2*x01^2 + 3/2*x00^2*x02^2 - x00*x02^3,
  6*x00^2*x01^2 - x01^4 - 6*x00^2*x02^2 + x02^4,
  x00^4 - x01^4 - 6*x00^2*x02^2 + 6*x01^2*x02^2,
  -3/2*x00^2*x01^2 + x00*x01^3 + 3/2*x00^2*x02^2 - x00*x02^3,
  3/2*x00^2*x02^2 - 3/2*x01^2*x02^2 - x00*x02^3 + x01*x02^3,
  x00^3*x02 - x00*x02^3),
 ((2,), (3,)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01)}

In [56]:
polarization_operators_by_multidegree(P)

{(-1, 1): [<functools.partial object at 0x7ff0d86f1418>],
 (1, -1): [<functools.partial object at 0x7ff0d858caa0>]}

In [52]:
P = DiagonalPolynomialRing(QQ, 3, 2)
polarization_operators_by_multidegree(P)

{(-1, 1): [<functools.partial object at 0x7ff0d86f1470>],
 (1, -1): [<functools.partial object at 0x7ff0d86f1af8>]}

In [83]:
def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]
    
def polarizedSpace(pol_ring, basis, operators):
    n = pol_ring._n
    generators = {}
    for key, value in basis.iteritems():
        generators[key] = [P(b) for b in value]
    return Subspace(generators, operators, add_degrees=add_degrees_isotypic)

In [85]:
list_steenrod = merge_dict(steenrod_op(P,1), steenrod_op(P,2))
op_pol = merge_dict(polarization_operators_by_multidegree(P), list_steenrod)
V2 = polarizedSpace(P, isotyp(V1, 3), op_pol)

In [86]:
V2.basis()

{((4,), (3,)): (x01^3*x02 - x01*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 - 2/3*x00^3*x02 + x00^2*x02^2,
  -x00^3*x02 + x00*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 + x01^2*x02^2 - 2/3*x01*x02^3,
  x00^3*x01 - x00*x01^3,
  -x00^4 + x01^4 + 4*x00^3*x02 - 4*x01*x02^3,
  -x00^4 + 4*x00*x01^3 - 4*x01*x02^3 + x02^4),
 ((2,), (1, 1, 1)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((6,),
  (1,
   1,
   1)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((3,), (3,)): (-1/3*x01^3 + x01*x02^2,
  x00^3 - x01^3,
  -x01^3 + x02^3,
  -1/3*x01^3 + x00*x02^2,
  -1/3*x01^3 + x00^2*x02,
  x00*x01^2 - 1/3*x01^3,
  x00^2*x01 - 1/3*x01^3,
  -1/3*x01^3 + x01^2*x02),
 ((2,), (2, 1)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((3,), (2, 1)): (-1/3*x01^3 + x01*x02^2,
  x00^3 - x01^3,
  -x01^3 + x02^3,
  -1/3*x01^3 + x00*x02^2,
  -1/3*x01^3 + x00^2*x02,
  x00*x01^2 - 1/3*x01^3,
  x00^2*x01 - 1/3*x01^3,
  -1/3*x01^3 + x01^2*x02),
 ((5,),
  (3,)): (-x00^3*x01^2 + x00^2*x01^3 - 1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^3*x02^2 - x01^3*x02^2, x00^4*x01 - 2*x00^3*x01^2 - x00^4*x02 + x01^4*x02 + 2*x00^3*x02^2 - x01*x02^4, -1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^2*x02^3 - x01^2*x02^3, 2*x00^3*x01^2 - x00*x01^4 - 2*x00^3*x02^2 + x00*x02^4),
 ((6,),
  (2,
   1)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((6,),
  (3,)): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,),
 ((3,), (1, 1, 1)): (-1/3*x01^3 + x01*x02^2,
  x00^3 - x01^3,
  -x01^3 + x02^3,
  -1/3*x01^3 + x00*x02^2,
  -1/3*x01^3 + x00^2*x02,
  x00*x01^2 - 1/3*x01^3,
  x00^2*x01 - 1/3*x01^3,
  -1/3*x01^3 + x01^2*x02),
 ((5,),
  (1,
   1,
   1)): (-x00^3*x01^2 + x00^2*x01^3 - 1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^3*x02^2 - x01^3*x02^2, x00^4*x01 - 2*x00^3*x01^2 - x00^4*x02 + x01^4*x02 + 2*x00^3*x02^2 - x01*x02^4, -1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^2*x02^3 - x01^2*x02^3, 2*x00^3*x01^2 - x00*x01^4 - 2*x00^3*x02^2 + x00*x02^4),
 ((5,),
  (2,
   1)): (-x00^3*x01^2 + x00^2*x01^3 - 1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^3*x02^2 - x01^3*x02^2, x00^4*x01 - 2*x00^3*x01^2 - x00^4*x02 + x01^4*x02 + 2*x00^3*x02^2 - x01*x02^4, -1/2*x00^4*x02 + 1/2*x01^4*x02 + x00^2*x02^3 - x01^2*x02^3, 2*x00^3*x01^2 - x00*x01^4 - 2*x00^3*x02^2 + x00*x02^4),
 ((0,), (1, 1, 1)): (1,),
 ((0,), (3,)): (1,),
 ((2,), (3,)): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 ((0,), (2, 1)): (1,),
 ((1,), (1, 1, 1)): (x00, x01, x02),
 ((1,), (3,)): (x00, x01, x02),
 ((4,), (1, 1, 1)): (x01^3*x02 - x01*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 - 2/3*x00^3*x02 + x00^2*x02^2,
  -x00^3*x02 + x00*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 + x01^2*x02^2 - 2/3*x01*x02^3,
  x00^3*x01 - x00*x01^3,
  -x00^4 + x01^4 + 4*x00^3*x02 - 4*x01*x02^3,
  -x00^4 + 4*x00*x01^3 - 4*x01*x02^3 + x02^4),
 ((4,), (2, 1)): (x01^3*x02 - x01*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 - 2/3*x00^3*x02 + x00^2*x02^2,
  -x00^3*x02 + x00*x02^3,
  -x00^2*x01^2 + 2/3*x00*x01^3 + x01^2*x02^2 - 2/3*x01*x02^3,
  x00^3*x01 - x00*x01^3,
  -x00^4 + x01^4 + 4*x00^3*x02 - 4*x01*x02^3,
  -x00^4 + 4*x00*x01^3 - 4*x01*x02^3 + x02^4),
 ((1,), (2, 1)): (x00, x01, x02)}

In [ ]:
def character(S, n, left_basis=s, right_basis=s, row_symmetry=None):
    basis = S.basis()
    charac = 0
    for nu in Partitions(n):
        basis_nu = {}
        for key, value in basis.iteritems():
            if key[1] == nu:
                basis_nu[key[0]] = value
            if row_symmetry == "permutation":
                #avec les monomiales
            else:
                #avec les q
    return charac